#  DSPy Multi-Agent Systems & Arquiteturas Cognitivas

**Versão:** 1.0 - Multi-Agent Cognitive Architectures  
**Nível:** Intermediário/Avançado  
**Tempo estimado:** 45-60 minutos  
**Abordagem:** Hands-On com Arquiteturas Avançadas

---

##  Sobre este Notebook

Este notebook explora **sistemas multi-agent** e **arquiteturas cognitivas** com DSPy.

Você vai:
-  Criar **múltiplos agentes especializados**
-  Implementar diferentes **arquiteturas cognitivas**
-  Entender **padrões de coordenação** entre agentes
-  Aplicar no domínio de **airline booking** com especialização
-  Comparar **performance** de diferentes arquiteturas

###  Pré-requisitos
- Notebook básico de DSPy ReAct completado
- Compreensão de agentes individuais
- Python intermediário

###  O que vamos cobrir

1. **Fundamentos Multi-Agent**
   - O que são sistemas multi-agent
   - Quando usar múltiplos agentes
   - Vantagens e desafios

2. **Arquiteturas Cognitivas**
   - Hierarchical (Coordenador + Especialistas)
   - Sequential/Pipeline (Cadeia de especialização)
   - Collaborative/Debate (Múltiplas perspectivas)
   - Reflexive/Self-Critique (Auto-melhoria)

3. **Implementação Prática**
   - Agentes especializados: Busca, Recomendação, Booking, Suporte
   - Coordenação e comunicação
   - Gerenciamento de estado compartilhado

---

Based on DSPy best practices and multi-agent patterns


##  Parte 1: Fundamentos de Multi-Agent Systems

### O que são Sistemas Multi-Agent?

Um **sistema multi-agent** é composto por múltiplos agentes autônomos que:
- Têm **responsabilidades específicas** (especialização)
- **Colaboram** para resolver problemas complexos
- Podem ter **arquiteturas cognitivas** diferentes

### Por que usar Multi-Agent?

 **Especialização**: Cada agente domina uma área específica  
 **Escalabilidade**: Fácil adicionar novos agentes/capacidades  
 **Manutenibilidade**: Mudanças isoladas por agente  
 **Qualidade**: Múltiplas perspectivas = melhores decisões  

### Quando usar?

-  Problema tem **múltiplos domínios** distintos
-  Necessidade de **expertise especializada**
-  Benefício de **múltiplas perspectivas**
-  Workflow pode ser **decomposto** em etapas

### Desafios

-  Coordenação entre agentes
-  Gerenciamento de estado compartilhado
-  Resolução de conflitos
-  Overhead de comunicação


## Setup e Imports


In [ ]:
import dspy
import os
from datetime import datetime, timedelta
from typing import List, Optional, Dict, Any, Tuple
from pydantic import BaseModel, Field
from dataclasses import dataclass
import json
import uuid
from enum import Enum
from dotenv import load_dotenv

load_dotenv()


In [ ]:
# Configurar LLM
lm = dspy.LM('openai/gpt-4o-mini')
# lm = dspy.LM('groq/llama-3.3-70b-versatile')
dspy.configure(lm=lm)


##  Parte 2: Arquiteturas Cognitivas - Visão Geral

### As 4 Arquiteturas que vamos implementar

#### 1. **Hierarchical (Hierárquica)** 
```
         [Coordinator]
              |
    
    v         v         v
[Search]  [Recommend] [Book]
```
- **Coordenador**: Decide qual especialista chamar
- **Especialistas**: Executam tarefas específicas
- **Uso**: Quando há clara hierarquia de decisão

#### 2. **Sequential/Pipeline (Sequencial)** 
```
[Search] → [Recommend] → [Book] → [Confirm]
```
- **Fluxo linear**: Cada agente passa resultado ao próximo
- **Especialização**: Cada etapa tem responsabilidade única
- **Uso**: Workflows com etapas bem definidas

#### 3. **Collaborative/Debate (Colaborativa)** 
```
[Agent A] ⇄ [Agent B] ⇄ [Agent C]
     ↓          ↓          ↓
         [Consensus]
```
- **Múltiplas perspectivas**: Agentes debatem solução
- **Consenso**: Decisão emerge da colaboração
- **Uso**: Decisões complexas com trade-offs

#### 4. **Reflexive/Self-Critique (Reflexiva)** 
```
[Actor] → [Critic] → [Refiner] → [Output]
              ↓           ↓
               (loop)
```
- **Auto-avaliação**: Agente critica próprio resultado
- **Refinamento iterativo**: Melhora através de feedback
- **Uso**: Quando qualidade é crítica


## Data Models para Multi-Agent


In [ ]:
# Models básicos
class UserProfile(BaseModel):
    name: str
    user_id: str
    email: str
    phone: str
    frequent_flyer_number: Optional[str] = None
    preferences: Dict[str, Any] = Field(default_factory=dict)

class Flight(BaseModel):
    flight_id: str
    flight_number: str
    departure_airport: str
    arrival_airport: str
    departure_time: str
    arrival_time: str
    duration_minutes: int
    price: float
    available_seats: int
    airline: str = "Default Airlines"
    aircraft_type: str = "Boeing 737"
    
class Itinerary(BaseModel):
    itinerary_id: str
    user_id: str
    flights: List[Flight]
    total_price: float
    booking_date: str
    status: str

# Models para comunicação entre agentes
class AgentRole(str, Enum):
    COORDINATOR = "coordinator"
    SEARCH = "search_specialist"
    RECOMMENDATION = "recommendation_specialist"
    BOOKING = "booking_specialist"
    SUPPORT = "support_specialist"

class AgentMessage(BaseModel):
    """Mensagem trocada entre agentes"""
    from_agent: AgentRole
    to_agent: AgentRole
    content: str
    data: Optional[Dict[str, Any]] = None
    timestamp: str = Field(default_factory=lambda: datetime.now().isoformat())

class AgentContext(BaseModel):
    """Contexto compartilhado entre agentes"""
    user_request: str
    user_profile: Optional[UserProfile] = None
    conversation_history: List[AgentMessage] = Field(default_factory=list)
    search_results: Optional[List[Flight]] = None
    selected_flight: Optional[Flight] = None
    booking_result: Optional[Dict[str, Any]] = None
    metadata: Dict[str, Any] = Field(default_factory=dict)


## Mock Database (Expandido)


In [ ]:
# Mock databases com mais dados
users_db = {
    "Adam": UserProfile(
        name="Adam",
        user_id="user_001",
        email="adam@example.com",
        phone="+1-555-0101",
        frequent_flyer_number="FF12345",
        preferences={
            "preferred_airlines": ["American Airlines", "United"],
            "seat_preference": "window",
            "priority": "price"
        }
    ),
    "Sarah": UserProfile(
        name="Sarah",
        user_id="user_002",
        email="sarah@example.com",
        phone="+1-555-0102",
        preferences={
            "preferred_airlines": ["Delta"],
            "seat_preference": "aisle",
            "priority": "duration"
        }
    )
}

flights_db = {
    "SFO-JFK": [
        Flight(
            flight_id="f001",
            flight_number="AA101",
            departure_airport="SFO",
            arrival_airport="JFK",
            departure_time="08:00",
            arrival_time="16:30",
            duration_minutes=330,
            price=450.00,
            available_seats=15,
            airline="American Airlines",
            aircraft_type="Boeing 777"
        ),
        Flight(
            flight_id="f002",
            flight_number="UA205",
            departure_airport="SFO",
            arrival_airport="JFK",
            departure_time="14:00",
            arrival_time="22:45",
            duration_minutes=345,
            price=380.00,
            available_seats=8,
            airline="United",
            aircraft_type="Boeing 787"
        ),
        Flight(
            flight_id="f003",
            flight_number="DL150",
            departure_airport="SFO",
            arrival_airport="JFK",
            departure_time="11:00",
            arrival_time="19:20",
            duration_minutes=320,
            price=520.00,
            available_seats=12,
            airline="Delta",
            aircraft_type="Airbus A350"
        )
    ],
    "JFK-LAX": [
        Flight(
            flight_id="f004",
            flight_number="DL302",
            departure_airport="JFK",
            arrival_airport="LAX",
            departure_time="10:00",
            arrival_time="13:30",
            duration_minutes=390,
            price=520.00,
            available_seats=12,
            airline="Delta",
            aircraft_type="Boeing 767"
        )
    ]
}

itineraries_db = {}
tickets_db = {}


##  Ferramentas (Tools) para os Agentes


In [ ]:
# Ferramentas base
def fetch_flight_info(departure: str, arrival: str, date: str) -> str:
    """Buscar voos disponíveis para uma rota e data."""
    route = f"{departure}-{arrival}"
    flights = flights_db.get(route, [])
    
    if not flights:
        return json.dumps({"error": f"No flights found for route {route}"})
    
    flights_data = [flight.model_dump() for flight in flights]
    return json.dumps({"flights": flights_data, "count": len(flights_data)})

def get_user_info(name: str) -> str:
    """Obter informações do perfil do usuário."""
    user = users_db.get(name)
    if not user:
        return json.dumps({"error": f"User {name} not found"})
    return json.dumps({"user": user.model_dump()})

def book_flight(user_name: str, flight_id: str, date: str) -> str:
    """Reservar um voo para um usuário."""
    user = users_db.get(user_name)
    if not user:
        return json.dumps({"error": f"User {user_name} not found"})
    
    flight = None
    for route_flights in flights_db.values():
        for f in route_flights:
            if f.flight_id == flight_id:
                flight = f
                break
        if flight:
            break
    
    if not flight:
        return json.dumps({"error": f"Flight {flight_id} not found"})
    
    if flight.available_seats <= 0:
        return json.dumps({"error": "No available seats"})
    
    itinerary_id = str(uuid.uuid4())
    confirmation_number = f"CONF{uuid.uuid4().hex[:8].upper()}"
    
    itinerary = Itinerary(
        itinerary_id=itinerary_id,
        user_id=user.user_id,
        flights=[flight],
        total_price=flight.price,
        booking_date=datetime.now().strftime("%Y-%m-%d"),
        status="confirmed"
    )
    
    itineraries_db[itinerary_id] = itinerary
    flight.available_seats -= 1
    
    return json.dumps({
        "success": True,
        "confirmation_number": confirmation_number,
        "itinerary_id": itinerary_id,
        "flight": flight.model_dump(),
        "total_price": flight.price
    })

# Ferramentas específicas para multi-agent
def analyze_user_preferences(user_name: str, flights_json: str) -> str:
    """Analisar preferências do usuário e ranquear voos."""
    user = users_db.get(user_name)
    if not user:
        return json.dumps({"error": "User not found"})
    
    flights_data = json.loads(flights_json)
    flights = [Flight(**f) for f in flights_data.get("flights", [])]
    
    preferences = user.preferences
    priority = preferences.get("priority", "price")
    
    if priority == "price":
        ranked = sorted(flights, key=lambda f: f.price)
    elif priority == "duration":
        ranked = sorted(flights, key=lambda f: f.duration_minutes)
    else:
        ranked = flights
    
    preferred_airlines = preferences.get("preferred_airlines", [])
    if preferred_airlines:
        preferred = [f for f in ranked if f.airline in preferred_airlines]
        others = [f for f in ranked if f.airline not in preferred_airlines]
        ranked = preferred + others
    
    return json.dumps({
        "ranked_flights": [f.model_dump() for f in ranked],
        "recommendation_reason": f"Ranked by {priority}, preferred airlines first",
        "top_recommendation": ranked[0].model_dump() if ranked else None
    })


##  Arquitetura 1: Hierarchical (Hierárquica)

### Conceito

Na arquitetura hierárquica, temos:
- **Coordenador**: Agente principal que analisa a requisição e delega para especialistas
- **Especialistas**: Agentes especializados em domínios específicos

```
         [Coordinator Agent]
              |
    
    v         v         v
[Search]  [Recommend] [Book]
```

### Quando usar?
-  Quando há domínios claramente separados
-  Quando precisa de roteamento inteligente
-  Quando especialização é importante

### Implementação


In [ ]:
# Signatures para agentes especializados

class SearchSpecialistSignature(dspy.Signature):
    """
    Especialista em busca de voos.
    Encontra voos disponíveis baseado em critérios do usuário.
    """
    user_request: str = dspy.InputField(desc="Requisição do usuário")
    departure: str = dspy.InputField(desc="Aeroporto de partida")
    arrival: str = dspy.InputField(desc="Aeroporto de chegada")
    date: str = dspy.InputField(desc="Data do voo")
    
    analysis: str = dspy.OutputField(desc="Análise da busca")
    flights_found: str = dspy.OutputField(desc="Voos encontrados em JSON")

class RecommendationSpecialistSignature(dspy.Signature):
    """
    Especialista em recomendações de voos.
    Analisa preferências do usuário e ranqueia voos.
    """
    user_request: str = dspy.InputField(desc="Requisição do usuário")
    user_name: str = dspy.InputField(desc="Nome do usuário")
    available_flights: str = dspy.InputField(desc="Voos disponíveis em JSON")
    
    analysis: str = dspy.OutputField(desc="Análise das preferências")
    recommendation: str = dspy.OutputField(desc="Recomendação final")

class BookingSpecialistSignature(dspy.Signature):
    """
    Especialista em reservas de voos.
    Realiza o booking e confirma a reserva.
    """
    user_request: str = dspy.InputField(desc="Requisição do usuário")
    user_name: str = dspy.InputField(desc="Nome do usuário")
    flight_id: str = dspy.InputField(desc="ID do voo a reservar")
    date: str = dspy.InputField(desc="Data do voo")
    
    booking_result: str = dspy.OutputField(desc="Resultado da reserva")
    confirmation: str = dspy.OutputField(desc="Mensagem de confirmação")

class CoordinatorSignature(dspy.Signature):
    """
    Coordenador que analisa a requisição e decide qual especialista chamar.
    """
    user_request: str = dspy.InputField(desc="Requisição do usuário")
    
    required_specialist: str = dspy.OutputField(
        desc="Especialista necessário: 'search', 'recommendation', 'booking', ou 'general'"
    )
    reasoning: str = dspy.OutputField(desc="Razão da escolha")


In [ ]:
# Implementação dos agentes especializados
class SearchSpecialist(dspy.Module):
    def __init__(self):
        super().__init__()
        self.predictor = dspy.ChainOfThought(SearchSpecialistSignature)
        
    def forward(self, user_request: str, departure: str, arrival: str, date: str):
        # Buscar voos
        flights_json = fetch_flight_info(departure, arrival, date)
        
        # Analisar resultados
        result = self.predictor(
            user_request=user_request,
            departure=departure,
            arrival=arrival,
            date=date
        )
        
        return dspy.Prediction(
            analysis=result.analysis,
            flights_found=flights_json
        )

class RecommendationSpecialist(dspy.Module):
    def __init__(self):
        super().__init__()
        self.predictor = dspy.ChainOfThought(RecommendationSpecialistSignature)
        
    def forward(self, user_request: str, user_name: str, available_flights: str):
        # Analisar preferências e ranquear
        recommendation_json = analyze_user_preferences(user_name, available_flights)
        
        # Gerar recomendação com análise
        result = self.predictor(
            user_request=user_request,
            user_name=user_name,
            available_flights=available_flights
        )
        
        return dspy.Prediction(
            analysis=result.analysis,
            recommendation=recommendation_json
        )

class BookingSpecialist(dspy.Module):
    def __init__(self):
        super().__init__()
        self.predictor = dspy.ChainOfThought(BookingSpecialistSignature)
        
    def forward(self, user_request: str, user_name: str, flight_id: str, date: str):
        # Realizar booking
        booking_result = book_flight(user_name, flight_id, date)
        
        # Gerar confirmação
        result = self.predictor(
            user_request=user_request,
            user_name=user_name,
            flight_id=flight_id,
            date=date
        )
        
        return dspy.Prediction(
            booking_result=booking_result,
            confirmation=result.confirmation
        )

# Sistema Hierarchical completo
class HierarchicalMultiAgent(dspy.Module):
    def __init__(self):
        super().__init__()
        self.coordinator = dspy.ChainOfThought(CoordinatorSignature)
        self.search_specialist = SearchSpecialist()
        self.recommendation_specialist = RecommendationSpecialist()
        self.booking_specialist = BookingSpecialist()
        
    def forward(self, user_request: str, **kwargs):
        # Coordenador decide qual especialista usar
        coordination = self.coordinator(user_request=user_request)
        
        specialist_type = coordination.required_specialist.lower()
        
        print(f" Coordinator Decision: {specialist_type}")
        print(f" Reasoning: {coordination.reasoning}\n")
        
        # Delegar para especialista apropriado
        if "search" in specialist_type:
            result = self.search_specialist(
                user_request=user_request,
                departure=kwargs.get("departure", "SFO"),
                arrival=kwargs.get("arrival", "JFK"),
                date=kwargs.get("date", "2025-12-01")
            )
            return dspy.Prediction(
                specialist="search",
                analysis=result.analysis,
                data=result.flights_found
            )
            
        elif "recommend" in specialist_type:
            result = self.recommendation_specialist(
                user_request=user_request,
                user_name=kwargs.get("user_name", "Adam"),
                available_flights=kwargs.get("available_flights", "{}")
            )
            return dspy.Prediction(
                specialist="recommendation",
                analysis=result.analysis,
                data=result.recommendation
            )
            
        elif "book" in specialist_type:
            result = self.booking_specialist(
                user_request=user_request,
                user_name=kwargs.get("user_name", "Adam"),
                flight_id=kwargs.get("flight_id", "f001"),
                date=kwargs.get("date", "2025-12-01")
            )
            return dspy.Prediction(
                specialist="booking",
                confirmation=result.confirmation,
                data=result.booking_result
            )
        
        return dspy.Prediction(
            specialist="general",
            message="Request handled by coordinator directly"
        )


###  Teste da Arquitetura Hierarchical


In [ ]:
# Teste da arquitetura hierarchical
hierarchical_system = HierarchicalMultiAgent()

# Teste 1: Busca de voos
print("=" * 60)
print("TESTE 1: Busca de Voos")
print("=" * 60)
result1 = hierarchical_system(
    user_request="I need to find flights from SFO to JFK on December 1st",
    departure="SFO",
    arrival="JFK",
    date="2025-12-01"
)
print(f"Specialist Used: {result1.specialist}")
print(f"Analysis: {result1.analysis}")
print(f"Data: {result1.data[:200]}...")  # Primeiros 200 caracteres

print("\n" + "=" * 60)
print("TESTE 2: Recomendação")
print("=" * 60)
# Primeiro buscar voos
flights_result = fetch_flight_info("SFO", "JFK", "2025-12-01")
result2 = hierarchical_system(
    user_request="Which flight should I choose for my trip? I prefer cheaper options.",
    user_name="Adam",
    available_flights=flights_result
)
print(f"Specialist Used: {result2.specialist}")
print(f"Analysis: {result2.analysis}")
print(f"Recommendation: {result2.data[:200]}...")

# NOTA: Descomente para testar booking (irá realmente criar uma reserva)
# print("\n" + "=" * 60)
# print("TESTE 3: Booking")
# print("=" * 60)
# result3 = hierarchical_system(
#     user_request="Please book flight f001 for me on December 1st",
#     user_name="Adam",
#     flight_id="f001",
#     date="2025-12-01"
# )
# print(f"Specialist Used: {result3.specialist}")
# print(f"Confirmation: {result3.confirmation}")
# print(f"Booking Data: {result3.data}")


##  Arquitetura 2: Sequential/Pipeline (Sequencial)

### Conceito

Na arquitetura sequencial, os agentes trabalham em uma cadeia:
- **Fluxo linear**: Cada agente processa e passa ao próximo
- **Transformação de dados**: Cada etapa refina o resultado
- **Especialização progressiva**: Do geral ao específico

```
[Search Agent] → [Analysis Agent] → [Recommendation Agent] → [Booking Agent]
     |               |                    |                      |
  "Find"         "Analyze"            "Recommend"           "Confirm"
```

### Quando usar?
-  Workflow tem etapas bem definidas
-  Cada etapa depende da anterior
-  Processo é naturalmente sequencial
-  Rastreabilidade é importante

### Implementação


In [ ]:
# Signatures para pipeline
class PipelineSearchSignature(dspy.Signature):
    """Etapa 1: Buscar voos disponíveis"""
    departure: str = dspy.InputField()
    arrival: str = dspy.InputField()
    date: str = dspy.InputField()
    
    search_summary: str = dspy.OutputField(desc="Resumo da busca")
    flights_json: str = dspy.OutputField(desc="Voos em JSON")

class PipelineAnalysisSignature(dspy.Signature):
    """Etapa 2: Analisar voos encontrados"""
    flights_json: str = dspy.InputField(desc="Voos da etapa anterior")
    user_name: str = dspy.InputField()
    
    analysis: str = dspy.OutputField(desc="Análise detalhada dos voos")
    key_insights: str = dspy.OutputField(desc="Insights principais")

class PipelineRecommendationSignature(dspy.Signature):
    """Etapa 3: Recomendar melhor opção"""
    analysis: str = dspy.InputField(desc="Análise da etapa anterior")
    flights_json: str = dspy.InputField()
    user_name: str = dspy.InputField()
    
    recommendation: str = dspy.OutputField(desc="Recomendação final")
    flight_id: str = dspy.OutputField(desc="ID do voo recomendado")

class PipelineConfirmationSignature(dspy.Signature):
    """Etapa 4: Confirmar e finalizar"""
    recommendation: str = dspy.InputField()
    flight_id: str = dspy.InputField()
    user_name: str = dspy.InputField()
    
    final_message: str = dspy.OutputField(desc="Mensagem final ao usuário")

# Implementação do Pipeline
class SequentialPipelineMultiAgent(dspy.Module):
    def __init__(self):
        super().__init__()
        self.search_agent = dspy.ChainOfThought(PipelineSearchSignature)
        self.analysis_agent = dspy.ChainOfThought(PipelineAnalysisSignature)
        self.recommendation_agent = dspy.ChainOfThought(PipelineRecommendationSignature)
        self.confirmation_agent = dspy.ChainOfThought(PipelineConfirmationSignature)
        
    def forward(self, departure: str, arrival: str, date: str, user_name: str):
        print(" PIPELINE EXECUTION")
        print("=" * 60)
        
        # Etapa 1: Search
        print(" Step 1: Searching for flights...")
        flights_json = fetch_flight_info(departure, arrival, date)
        search_result = self.search_agent(
            departure=departure,
            arrival=arrival,
            date=date
        )
        print(f" Found: {search_result.search_summary}")
        print()
        
        # Etapa 2: Analysis
        print(" Step 2: Analyzing options...")
        analysis_result = self.analysis_agent(
            flights_json=flights_json,
            user_name=user_name
        )
        print(f" Analysis: {analysis_result.key_insights[:100]}...")
        print()
        
        # Etapa 3: Recommendation
        print(" Step 3: Generating recommendation...")
        recommendation_result = self.recommendation_agent(
            analysis=analysis_result.analysis,
            flights_json=flights_json,
            user_name=user_name
        )
        print(f" Recommended: Flight {recommendation_result.flight_id}")
        print()
        
        # Etapa 4: Confirmation
        print(" Step 4: Preparing final message...")
        confirmation_result = self.confirmation_agent(
            recommendation=recommendation_result.recommendation,
            flight_id=recommendation_result.flight_id,
            user_name=user_name
        )
        print(f" Ready to present to user")
        print("=" * 60)
        
        return dspy.Prediction(
            search_summary=search_result.search_summary,
            analysis=analysis_result.analysis,
            recommendation=recommendation_result.recommendation,
            flight_id=recommendation_result.flight_id,
            final_message=confirmation_result.final_message,
            flights_data=flights_json
        )


In [ ]:
# Teste da arquitetura Sequential/Pipeline
pipeline_system = SequentialPipelineMultiAgent()

result = pipeline_system(
    departure="SFO",
    arrival="JFK",
    date="2025-12-01",
    user_name="Adam"
)

print("\n RESULTADO FINAL")
print("=" * 60)
print(f"Final Message: {result.final_message}")
print(f"Recommended Flight: {result.flight_id}")


##  Arquitetura 3: Collaborative/Debate (Colaborativa)

### Conceito

Na arquitetura colaborativa, múltiplos agentes debatem para chegar à melhor solução:
- **Múltiplas perspectivas**: Cada agente tem visão diferente
- **Debate**: Agentes argumentam e contra-argumentam
- **Consenso**: Solução emerge da colaboração

```
[Price Optimizer] ⇄ [Comfort Advisor] ⇄ [Time Optimizer]
       |                  |                    |
       
                          ↓
                   [Consensus Agent]
```

### Quando usar?
-  Decisões complexas com trade-offs
-  Múltiplas perspectivas agregam valor
-  Não há resposta obviamente "correta"
-  Qualidade > Velocidade

### Implementação


In [ ]:
# Signatures para agentes colaborativos
class PriceOptimizerSignature(dspy.Signature):
    """Agente focado em otimizar preço"""
    flights_json: str = dspy.InputField()
    user_context: str = dspy.InputField()
    other_opinions: str = dspy.InputField(desc="Opiniões de outros agentes")
    
    price_analysis: str = dspy.OutputField()
    recommendation: str = dspy.OutputField()
    flight_id: str = dspy.OutputField()

class ComfortAdvisorSignature(dspy.Signature):
    """Agente focado em conforto e qualidade"""
    flights_json: str = dspy.InputField()
    user_context: str = dspy.InputField()
    other_opinions: str = dspy.InputField(desc="Opiniões de outros agentes")
    
    comfort_analysis: str = dspy.OutputField()
    recommendation: str = dspy.OutputField()
    flight_id: str = dspy.OutputField()

class TimeOptimizerSignature(dspy.Signature):
    """Agente focado em otimizar tempo de viagem"""
    flights_json: str = dspy.InputField()
    user_context: str = dspy.InputField()
    other_opinions: str = dspy.InputField(desc="Opiniões de outros agentes")
    
    time_analysis: str = dspy.OutputField()
    recommendation: str = dspy.OutputField()
    flight_id: str = dspy.OutputField()

class ConsensusSignature(dspy.Signature):
    """Agente que sintetiza opiniões e forma consenso"""
    price_opinion: str = dspy.InputField()
    comfort_opinion: str = dspy.InputField()
    time_opinion: str = dspy.InputField()
    flights_json: str = dspy.InputField()
    
    final_analysis: str = dspy.OutputField()
    consensus_recommendation: str = dspy.OutputField()
    selected_flight_id: str = dspy.OutputField()

# Implementação Collaborative
class CollaborativeDebateMultiAgent(dspy.Module):
    def __init__(self, debate_rounds: int = 2):
        super().__init__()
        self.debate_rounds = debate_rounds
        self.price_optimizer = dspy.ChainOfThought(PriceOptimizerSignature)
        self.comfort_advisor = dspy.ChainOfThought(ComfortAdvisorSignature)
        self.time_optimizer = dspy.ChainOfThought(TimeOptimizerSignature)
        self.consensus_agent = dspy.ChainOfThought(ConsensusSignature)
        
    def forward(self, departure: str, arrival: str, date: str, user_name: str):
        print(" COLLABORATIVE DEBATE")
        print("=" * 60)
        
        # Buscar voos
        flights_json = fetch_flight_info(departure, arrival, date)
        user = users_db.get(user_name)
        user_context = f"User: {user_name}, Preferences: {user.preferences if user else 'none'}"
        
        # Debate em múltiplas rodadas
        previous_opinions = "No previous opinions yet."
        
        for round_num in range(self.debate_rounds):
            print(f"\n DEBATE ROUND {round_num + 1}/{self.debate_rounds}")
            print("-" * 60)
            
            # Cada agente dá sua opinião
            print(" Price Optimizer analyzing...")
            price_result = self.price_optimizer(
                flights_json=flights_json,
                user_context=user_context,
                other_opinions=previous_opinions
            )
            
            print(" Comfort Advisor analyzing...")
            comfort_result = self.comfort_advisor(
                flights_json=flights_json,
                user_context=user_context,
                other_opinions=previous_opinions
            )
            
            print("⏱ Time Optimizer analyzing...")
            time_result = self.time_optimizer(
                flights_json=flights_json,
                user_context=user_context,
                other_opinions=previous_opinions
            )
            
            # Atualizar opiniões para próxima rodada
            previous_opinions = f"""
            Price Optimizer: {price_result.recommendation} (Flight: {price_result.flight_id})
            Comfort Advisor: {comfort_result.recommendation} (Flight: {comfort_result.flight_id})
            Time Optimizer: {time_result.recommendation} (Flight: {time_result.flight_id})
            """
            
            print(f"\n Round {round_num + 1} Recommendations:")
            print(f"   Price: Flight {price_result.flight_id}")
            print(f"   Comfort: Flight {comfort_result.flight_id}")
            print(f"  ⏱ Time: Flight {time_result.flight_id}")
        
        # Formar consenso
        print("\n FORMING CONSENSUS")
        print("-" * 60)
        consensus = self.consensus_agent(
            price_opinion=f"{price_result.price_analysis}\nRecommendation: {price_result.recommendation}",
            comfort_opinion=f"{comfort_result.comfort_analysis}\nRecommendation: {comfort_result.recommendation}",
            time_opinion=f"{time_result.time_analysis}\nRecommendation: {time_result.recommendation}",
            flights_json=flights_json
        )
        
        print(f" Consensus Reached!")
        print(f"Selected Flight: {consensus.selected_flight_id}")
        print("=" * 60)
        
        return dspy.Prediction(
            price_opinion=price_result,
            comfort_opinion=comfort_result,
            time_opinion=time_result,
            final_analysis=consensus.final_analysis,
            recommendation=consensus.consensus_recommendation,
            selected_flight_id=consensus.selected_flight_id
        )


In [ ]:
# Teste da arquitetura Collaborative/Debate
collaborative_system = CollaborativeDebateMultiAgent(debate_rounds=2)

result = collaborative_system(
    departure="SFO",
    arrival="JFK",
    date="2025-12-01",
    user_name="Sarah"  # Sarah prefere Delta e duração
)

print("\n RESULTADO FINAL DO DEBATE")
print("=" * 60)
print(f"Final Consensus: {result.recommendation}")
print(f"Selected Flight: {result.selected_flight_id}")


##  Arquitetura 4: Reflexive/Self-Critique (Reflexiva)

### Conceito

Na arquitetura reflexiva, o sistema se auto-avalia e refina:
- **Actor**: Gera resposta inicial
- **Critic**: Avalia e critica a resposta
- **Refiner**: Melhora baseado na crítica
- **Loop iterativo**: Continua até atingir qualidade

```
[Actor] → [Critic] → [Refiner] → [Output]
             ↓           ↓
              (feedback loop)
```

### Quando usar?
-  Qualidade é crítica
-  Tempo não é limitação
-  Benefício de múltiplas iterações
-  Auto-melhoria é desejável

### Implementação


In [ ]:
# Signatures para arquitetura reflexiva
class ActorSignature(dspy.Signature):
    """Agente que gera recomendação inicial"""
    flights_json: str = dspy.InputField()
    user_name: str = dspy.InputField()
    user_context: str = dspy.InputField()
    previous_critique: str = dspy.InputField(desc="Crítica da iteração anterior (vazio na primeira)")
    
    analysis: str = dspy.OutputField()
    recommendation: str = dspy.OutputField()
    flight_id: str = dspy.OutputField()
    reasoning: str = dspy.OutputField()

class CriticSignature(dspy.Signature):
    """Agente que critica a recomendação"""
    recommendation: str = dspy.InputField()
    reasoning: str = dspy.InputField()
    flights_json: str = dspy.InputField()
    user_context: str = dspy.InputField()
    
    critique: str = dspy.OutputField(desc="Crítica detalhada da recomendação")
    quality_score: str = dspy.OutputField(desc="Score de qualidade de 1-10")
    improvement_suggestions: str = dspy.OutputField(desc="Sugestões de melhoria")

class RefinerSignature(dspy.Signature):
    """Agente que refina baseado na crítica"""
    original_recommendation: str = dspy.InputField()
    critique: str = dspy.InputField()
    improvement_suggestions: str = dspy.InputField()
    flights_json: str = dspy.InputField()
    
    refined_analysis: str = dspy.OutputField()
    refined_recommendation: str = dspy.OutputField()
    refined_flight_id: str = dspy.OutputField()

# Implementação Reflexive
class ReflexiveSelfCritiqueMultiAgent(dspy.Module):
    def __init__(self, max_iterations: int = 3, quality_threshold: float = 8.0):
        super().__init__()
        self.max_iterations = max_iterations
        self.quality_threshold = quality_threshold
        self.actor = dspy.ChainOfThought(ActorSignature)
        self.critic = dspy.ChainOfThought(CriticSignature)
        self.refiner = dspy.ChainOfThought(RefinerSignature)
        
    def forward(self, departure: str, arrival: str, date: str, user_name: str):
        print(" REFLEXIVE SELF-CRITIQUE")
        print("=" * 60)
        
        # Buscar voos e contexto
        flights_json = fetch_flight_info(departure, arrival, date)
        user = users_db.get(user_name)
        user_context = f"User: {user_name}, Preferences: {user.preferences if user else 'none'}"
        
        previous_critique = "No previous critique"
        current_recommendation = None
        current_reasoning = None
        
        for iteration in range(self.max_iterations):
            print(f"\n ITERATION {iteration + 1}/{self.max_iterations}")
            print("-" * 60)
            
            # Step 1: Actor gera (ou refina) recomendação
            print(" Actor: Generating recommendation...")
            actor_result = self.actor(
                flights_json=flights_json,
                user_name=user_name,
                user_context=user_context,
                previous_critique=previous_critique
            )
            current_recommendation = actor_result.recommendation
            current_reasoning = actor_result.reasoning
            current_flight_id = actor_result.flight_id
            
            print(f"   Recommended: Flight {current_flight_id}")
            
            # Step 2: Critic avalia
            print(" Critic: Evaluating recommendation...")
            critic_result = self.critic(
                recommendation=current_recommendation,
                reasoning=current_reasoning,
                flights_json=flights_json,
                user_context=user_context
            )
            
            # Extrair quality score
            try:
                quality_score = float(critic_result.quality_score.split()[0])
            except:
                quality_score = 7.0  # Default se não conseguir extrair
            
            print(f"   Quality Score: {quality_score}/10")
            print(f"   Critique: {critic_result.critique[:80]}...")
            
            # Verificar se atingiu threshold
            if quality_score >= self.quality_threshold:
                print(f"\n Quality threshold reached! ({quality_score} >= {self.quality_threshold})")
                break
            
            # Step 3: Refiner melhora baseado na crítica
            if iteration < self.max_iterations - 1:  # Não refinar na última iteração
                print(" Refiner: Improving recommendation...")
                refiner_result = self.refiner(
                    original_recommendation=current_recommendation,
                    critique=critic_result.critique,
                    improvement_suggestions=critic_result.improvement_suggestions,
                    flights_json=flights_json
                )
                
                # Preparar crítica para próxima iteração
                previous_critique = f"""
                Previous Critique: {critic_result.critique}
                Quality Score: {quality_score}/10
                Improvements Suggested: {critic_result.improvement_suggestions}
                Refined Recommendation: {refiner_result.refined_recommendation}
                """
            else:
                print("   (Final iteration, no refinement needed)")
        
        print("\n" + "=" * 60)
        print(f" FINAL RESULT after {iteration + 1} iteration(s)")
        print(f"Quality Score: {quality_score}/10")
        print("=" * 60)
        
        return dspy.Prediction(
            analysis=actor_result.analysis,
            recommendation=current_recommendation,
            flight_id=current_flight_id,
            reasoning=current_reasoning,
            final_critique=critic_result.critique,
            quality_score=quality_score,
            iterations_used=iteration + 1
        )


In [ ]:
# Teste da arquitetura Reflexive/Self-Critique
reflexive_system = ReflexiveSelfCritiqueMultiAgent(max_iterations=3, quality_threshold=8.5)

result = reflexive_system(
    departure="SFO",
    arrival="JFK",
    date="2025-12-01",
    user_name="Adam"
)

print("\n RESULTADO FINAL")
print("=" * 60)
print(f"Iterations Used: {result.iterations_used}")
print(f"Quality Score: {result.quality_score}/10")
print(f"Recommended Flight: {result.flight_id}")
print(f"Reasoning: {result.reasoning}")
print(f"\nFinal Critique: {result.final_critique[:200]}...")


##  Comparação das Arquiteturas

### Resumo das 4 Arquiteturas

| Arquitetura | Complexidade | Velocidade | Qualidade | Melhor Para |
|-------------|--------------|------------|-----------|-------------|
| **Hierarchical** |  |  |  | Domínios bem separados, roteamento claro |
| **Sequential** |  |  |  | Workflows lineares, rastreabilidade |
| **Collaborative** |  |  |  | Decisões complexas, trade-offs |
| **Reflexive** |  |  |  | Qualidade crítica, auto-melhoria |

###  Quando usar cada uma?

#### 1. Hierarchical
**Use quando:**
-  Domínios claramente separados
-  Roteamento inteligente necessário
-  Velocidade é importante
-  Especialização por domínio

**Exemplo:** Sistema de atendimento ao cliente com departamentos

#### 2. Sequential/Pipeline
**Use quando:**
-  Processo tem etapas bem definidas
-  Cada etapa depende da anterior
-  Rastreabilidade é crítica
-  Transformação progressiva de dados

**Exemplo:** Pipeline de processamento de dados, ETL

#### 3. Collaborative/Debate
**Use quando:**
-  Múltiplas perspectivas agregam valor
-  Decisões têm trade-offs complexos
-  Consenso é melhor que decisão individual
-  Qualidade > Velocidade

**Exemplo:** Decisões estratégicas, análise de investimentos

#### 4. Reflexive/Self-Critique
**Use quando:**
-  Qualidade é absolutamente crítica
-  Benefício de refinamento iterativo
-  Auto-avaliação melhora resultados
-  Tempo não é limitação crítica

**Exemplo:** Geração de conteúdo crítico, revisão de código

###  Insights Importantes

1. **Não há "melhor" arquitetura** - depende do contexto
2. **Combine arquiteturas** - hierarchical pode usar reflexive internamente
3. **Trade-off qualidade vs velocidade** - sempre presente
4. **Custo aumenta com complexidade** - mais agentes = mais tokens
5. **DSPy permite implementar qualquer arquitetura** - framework flexível


##  Implementando Arquiteturas From Scratch

### Como implementar qualquer arquitetura cognitiva?

DSPy oferece os **building blocks** fundamentais para criar qualquer arquitetura:

#### 1. **Componentes Base do DSPy**
```python
# Signatures - Define o que o agente faz
class MyAgentSignature(dspy.Signature):
    input_field = dspy.InputField(desc="...")
    output_field = dspy.OutputField(desc="...")

# Modules - Implementa a lógica
class MyAgent(dspy.Module):
    def __init__(self):
        self.predictor = dspy.ChainOfThought(MyAgentSignature)
    
    def forward(self, **kwargs):
        return self.predictor(**kwargs)
```

#### 2. **Padrões de Comunicação**
```python
# Sequencial: A → B → C
result_a = agent_a(input)
result_b = agent_b(result_a.output)
result_c = agent_c(result_b.output)

# Paralelo: A, B, C → Combine
results = [agent_a(input), agent_b(input), agent_c(input)]
final = combiner(results)

# Loop: A → B → (critério) → A
while not condition:
    result = agent_a(result)
    critique = agent_b(result)
```

#### 3. **Estado Compartilhado**
```python
class SharedContext:
    def __init__(self):
        self.state = {}
    
    def update(self, key, value):
        self.state[key] = value
    
    def get(self, key):
        return self.state.get(key)

# Agentes acessam contexto compartilhado
context = SharedContext()
agent_a(input, context=context)
agent_b(context=context)
```

###  Outras Arquiteturas Possíveis

#### 1. **Star/Hub (Estrela)**
```
      [Hub Agent]
      /  |  |  \
    A1  A2 A3  A4
```
- Hub coordena múltiplos especialistas
- Similar a hierarchical mas mais centralizado

#### 2. **Tree/Hierarchical Deep**
```
        [Root]
       /      \
    [L1-A]  [L1-B]
     / \      / \
   A1  A2   B1  B2
```
- Hierarquia profunda
- Decomposição recursiva

#### 3. **Graph/Network**
```
A1 ⇄ A2 ⇄ A3
 ↓  ×  ↓  ×  ↓
A4 ⇄ A5 ⇄ A6
```
- Comunicação livre entre agentes
- Emergência de soluções

#### 4. **Marketplace/Bidding**
```
[Task] → [Bid A1] [Bid A2] [Bid A3] → [Winner Executes]
```
- Agentes competem por tarefas
- Auto-organização

###  Implementação From Scratch

```python
# Exemplo: Arquitetura de Marketplace
class MarketplaceArchitecture(dspy.Module):
    def __init__(self, agents: List[dspy.Module]):
        super().__init__()
        self.agents = agents
        self.auction = dspy.ChainOfThought(AuctionSignature)
    
    def forward(self, task: str):
        # 1. Cada agente faz uma proposta
        bids = []
        for agent in self.agents:
            bid = agent.propose(task)
            bids.append(bid)
        
        # 2. Auction decide o vencedor
        winner = self.auction(task=task, bids=bids)
        
        # 3. Vencedor executa
        result = self.agents[winner.agent_id].execute(task)
        
        return result
```

###  Princípios para Design de Arquiteturas

1. **Modularidade**: Agentes independentes e reutilizáveis
2. **Comunicação Clara**: Protocolos bem definidos
3. **Estado Explícito**: Contexto compartilhado transparente
4. **Composição**: Arquiteturas dentro de arquiteturas
5. **Observabilidade**: Rastreie tudo!

###  Dicas Práticas

- **Start Simple**: Comece com hierarchical ou sequential
- **Iterate**: Adicione complexidade conforme necessário
- **Measure**: Sempre meça qualidade e custo
- **Document**: Documente fluxo de comunicação
- **Test**: Teste cada agente isoladamente primeiro


##  Conclusão e Próximos Passos

###  O que você aprendeu

Neste notebook, você dominou:

 **Fundamentos Multi-Agent**
- Quando e por que usar múltiplos agentes
- Vantagens e desafios de sistemas multi-agent

 **4 Arquiteturas Cognitivas Principais**
- **Hierarchical**: Coordenador + Especialistas
- **Sequential**: Pipeline de processamento
- **Collaborative**: Debate e consenso
- **Reflexive**: Auto-crítica e refinamento

 **Implementação Prática**
- Como criar agentes especializados com DSPy
- Padrões de comunicação entre agentes
- Gerenciamento de estado e contexto

 **Design de Arquiteturas**
- Princípios para criar qualquer arquitetura
- Trade-offs entre diferentes abordagens
- Como combinar arquiteturas

###  Próximos Passos

#### 1. **Experimente e Modifique**
- Altere os parâmetros das arquiteturas
- Combine arquiteturas (ex: hierarchical com reflexive)
- Adicione novos agentes especializados

#### 2. **Aplique em Seu Domínio**
- Identifique problema que beneficia de multi-agent
- Escolha arquitetura apropriada
- Implemente incrementalmente

#### 3. **Otimize Performance**
- Use técnicas de otimização do DSPy (BootstrapFewShot, etc)
- Meça custos e latência
- Balance qualidade vs velocidade

#### 4. **Explore Arquiteturas Avançadas**
- Implemente Graph/Network architecture
- Teste Marketplace/Bidding
- Crie arquitetura híbrida customizada

#### 5. **Produção**
- Adicione observabilidade robusta (Langfuse, Arize)
- Implemente caching e retry logic
- Monitore custos e performance

###  Recursos Adicionais

**DSPy:**
- [Documentação Oficial](https://dspy.ai)
- [Paper DSPy](https://arxiv.org/abs/2310.03714)
- [GitHub DSPy](https://github.com/stanfordnlp/dspy)

**Multi-Agent Systems:**
- [Paper ReAct](https://arxiv.org/abs/2210.03629)
- [Reflexion Paper](https://arxiv.org/abs/2303.11366)
- [AutoGen Framework](https://github.com/microsoft/autogen)

**Arquiteturas Cognitivas:**
- [Cognitive Architectures for AI](https://www.cogarch.org/)
- [SOAR Architecture](http://soar.eecs.umich.edu/)

###  Dicas Finais

1. **Start Simple**: Não comece com collaborative de 10 agentes
2. **Measure Everything**: Custos, latência, qualidade
3. **Iterate**: Melhore arquitetura baseado em resultados
4. **Document**: Fluxo de comunicação é complexo
5. **Test Independently**: Cada agente deve funcionar sozinho

###  Obrigado!

Esperamos que este notebook tenha expandido sua compreensão sobre sistemas multi-agent e arquiteturas cognitivas com DSPy!

**Série de Notebooks DSPy:**
1. [Fundamentos Básicos (Linear)](dspy_agents_basic_linear_final.ipynb)
2. [Fundamentos Básicos (Hands-On)](dspy_agents_basic_handson_final.ipynb)
3. [Otimização Avançada (Linear)](dspy_agents_advanced_linear_final.ipynb)
4. [Otimização Avançada (Hands-On)](dspy_agents_advanced_handson_final.ipynb)
5. **→ Multi-Agent & Arquiteturas Cognitivas** (você está aqui!)

---

**Happy Building! **

Se tiver dúvidas ou sugestões, abra uma issue no repositório!
